In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of the dataset in characters: ", len(text))

length of the dataset in characters:  1115394


In [4]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# here are all the unique characters in the dataset
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# create a mapping from characters to integers
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for ch, i in stoi.items()}
encode = lambda s: [stoi[ch]for ch in s]
decode = lambda l: [itos[i] for i in l] # decoder: take a list of integers, output a string

print(encode('hello'))
print(decode(encode('hello')))

[46, 43, 50, 50, 53]
['h', 'e', 'l', 'l', 'o']


In [7]:
# Let's now encode the entire dataset into a tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.type)
print(data[:1000])

torch.Size([1115394]) <built-in method type of Tensor object at 0x117b208c0>
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15,

In [8]:
# Let's now split the data into training and validation sets
n = int(len(data)*0.9)
train_data = data[:n]
val_data = data[n:] # I think there is a more proper way to do this, but this is fine for now

In [9]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(context, "->", target)

tensor([18]) -> tensor(47)
tensor([18, 47]) -> tensor(56)
tensor([18, 47, 56]) -> tensor(57)
tensor([18, 47, 56, 57]) -> tensor(58)
tensor([18, 47, 56, 57, 58]) -> tensor(1)
tensor([18, 47, 56, 57, 58,  1]) -> tensor(15)
tensor([18, 47, 56, 57, 58,  1, 15]) -> tensor(47)
tensor([18, 47, 56, 57, 58,  1, 15, 47]) -> tensor(58)


In [11]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data         # select the data source
    ix = torch.randint(len(data)-block_size, (batch_size,))     # sample batch_size starting indices/charecter
    x = torch.stack([data[i   : i+block_size]   for i in ix])   # stack the charecters to make input - (batch_size, block_size)
    y = torch.stack([data[i+1 : i+1+block_size] for i in ix])   # stack the charecters to make target, shifted by 1 - (batch_size, block_size)

    return x, y

xb, yb = get_batch('train')
print("inputs")
print(xb.shape)
print(xb)
print("outputs")
print(yb.shape)
print(yb)

print('-----')

# (batch_size, block_size, block_size)
for b in range (batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f'{context} -> {target}')

inputs
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
outputs
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----
tensor([24]) -> 43
tensor([24, 43]) -> 58
tensor([24, 43, 58]) -> 5
tensor([24, 43, 58,  5]) -> 57
tensor([24, 43, 58,  5, 57]) -> 1
tensor([24, 43, 58,  5, 57,  1]) -> 46
tensor([24, 43, 58,  5, 57,  1, 46]) -> 43
tensor([24, 43, 58,  5, 57,  1, 46, 43]) -> 39
tensor([44]) -> 53
tensor([44, 53]) -> 56
tensor([44, 53, 56]) -> 1
tensor([44, 53, 56,  1]) -> 58
tensor([44, 53, 56,  1, 58]) -> 46
tensor([44, 53, 56,  1, 58, 46]) -> 39
tensor([44, 53, 56,  1, 58, 46, 39]) -> 58
tensor([44, 53, 56,  1, 58, 46, 39, 58]) -> 1
tensor([52]) -> 58
tensor([52, 58]) -> 1
tensor([52, 58,  1]) -> 58
tensor([52,

In [12]:
print(xb) # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__() # to initialize the nn.Module base class
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # initialize a matrix of shape (vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C), (batch_size, block_size, vocab_size)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C) # since we don't want to deal with (B,C,T) tensors expected for cross_entropy. we instead flatten the first two dimensions
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            # 

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idk is (B,T) array of indeces in the current context
        for _ in range(max_new_tokens):
            logits, loss = self(idx) # get the predictions
            logits = logits[:, -1, :] # focus only on the last time step - become (B,C)
            probs = F.softmax(logits, dim=-1) # apply softmax to get the probabilities - (B,C)
            idx_next = torch.multinomial(probs, num_samples=1) # sample from the distribution - (B,1)
            idx = torch.cat((idx, idx_next), dim=1) # append sample index to the running sequence - (B, T+1)
        return idx
    
    
# In the current example there is a lot of wasted compute in generate as we are computing logits for all tokens, instead of just the last one.
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print("".join(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist())))


torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [14]:
# create a Pytorch optimizer
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)

In [15]:
batch_size = 32

for steps in range(100): # increase the number of steps for better results
    xb, yb = get_batch('train') # sample a batch of data

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward() # gets the gradients
    optimizer.step() # updates the parameters

print(loss.item())

4.657352447509766


In [16]:
print(''.join(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist())))


oTo.JUZ!!zqe!
xBP qbs$Gy'AcOmrLwwt
p$x;Seh-onQbfM?OjKbn'NwUAW -Np3fkz$FVwAUEa-wfgC -wQo-R!v -Mj?,SPiTyZ;o-opr$mOiPJEYD-CfigkzD3p3?zvS;ADz;.y?o,ivCuC'zqHxcVT cHA
rT'Fd,SBMZyOslg!NXeF$sBe,juUzLq?w-wzP-h
ERjjxlgJzPbHxf$ q,q,KCDCU fqBOQT
SV&CW:xSVwZv'DG'NSPypDhKStKzC -$hslxIVzoivnp ,ethA:NCCGoi
tN!ljjP3fwJMwNelgUzzPGJlgihJ!d?q.d
pSPYgCuCJrIFtb
jQXg
pA.P LP,SPJi
DBcuBM:CixjJ$Jzkq,OLf3KLQLMGph$O 3DfiPHnXKuHMlyjxEiyZib3FaHV-oJa!zoc'XSP :CKGUhd?lgCOF$;;DTHZMlvvcmZAm;:iv'MMgO&Ywbc;BLCUd&vZINLIzkuTGZa
D.?


# The Mathematical Trick in Self-Attention

In [17]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
print('a=')
print(a)
print('--')
a = a / torch.sum(a, 1, keepdim=True) # the column of sums is duplicated for each columns
b = torch.randint(0,10, (3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
--
a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [18]:
# consider the following toy example:
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape
print(x)

tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])


In [19]:
# In these example we simply want to compute the mean of all the previous tokens at each time step t
# Later, instead of computing the mean, we will compute the attention over the previous tokens

# Version 1: compute the mean using for loop
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C)) # bow = bag of words, stands for mean of words/chars
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # t,C - the previous tokens - it increases by one at each step
        xbow[b,t] = torch.mean(xprev, dim=0) # C=2 - mean over the time dimension
        # print(xprev)
        # print(xbow[b,t])
        # print('---')
        
    # break
# print(xbow.shape)
# print(xbow)


In [20]:
# Version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones((T,T)))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # we weighted/averaged it by the previous tokens - later we will update using attention of the previous tokens
torch.allclose(xbow, xbow2)

True

In [21]:
# Version: use softmax
# Instead of starting with a matrix of ones, we start with a matrix of zeros and then fill the lower triangular part with -inf
# We use the softmax function to "normalize" the weights
# Hence, wei value can now adjust to the importance of the previous tokens soon. 
tril = torch.tril(torch.ones((T,T)))
wei = torch.zeros((T,T))
print(tril)
wei = wei.masked_fill(tril==0, float('-inf')) 
print(wei)
wei = F.softmax(wei, dim=-1)
print(wei)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])
tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000,

True

In [22]:
# Version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32          # batch, time, channels - channels is the embedding size = vectors that represent the token/character
x = torch.randn(B,T,C)  # B,T,C - word embedding - tokens in vectors - inputs

# Single head performing a self-attention
head_size = 16 # headsize is the available number of neurons for communication for each token

# learnable weights/function for the embedding/channels - afaik we don't clearly know yet what is happening in this weight and how it forms the needed features
query = nn.Linear(C, head_size, bias=False) # value to compare to key
key   = nn.Linear(C, head_size, bias=False) # value to compare to query
value = nn.Linear(C, head_size, bias=False) # weigths/function to produce the appropriate update value

# components of attention matrix
q = query(x)    # (B,T,H) - "this is what I need" because token values of header size will be compared to k
k = key(x)      # (B,T,H) - "this is what I am"   because token values of header size will be compared to q
                # k differs from q because of the shape being transposed which results to (B,H,T). Which enable tril to kills the future tokens properly

wei = q @ k.transpose(-2, -1)   # (B, T, 16) @ (B, 16, T) --> (B, T, T) - whole attention matrix - type: logits
                                # combination of q and k - where as both q & k become increasingly n>0 and same sign (the more similar they are) the more attention
                                # if not same sign, means more ignored.

tril = torch.tril(torch.ones((T,T)))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1) # "only able to see current and previous tokens attention matrix" - type: probabilities

v = value(x)    # (B,T,H) - how much to update x to reflect the attention from the previous tokens
out = wei @ v   # (B,T,T) @ (B,T,H) --> (B,T,H) - "scale v base on the attention matrix"

out.shape

torch.Size([4, 8, 16])

In [23]:
out[0]

tensor([[-0.1571,  0.8801,  0.1615, -0.7824, -0.1429,  0.7468,  0.1007, -0.5239,
         -0.8873,  0.1907,  0.1762, -0.5943, -0.4812, -0.4860,  0.2862,  0.5710],
        [ 0.2507,  0.1815, -0.0388, -0.2458, -0.1356,  0.2369, -0.1588, -0.3209,
         -0.4772,  0.4530,  0.4388, -0.3604, -0.0859, -0.0803,  0.1115,  0.9138],
        [ 0.3288,  0.0950, -0.1875, -0.0916, -0.0079,  0.0883, -0.0678, -0.1830,
         -0.4008,  0.0761,  0.3542, -0.1453, -0.1970, -0.0976,  0.0109,  1.0278],
        [ 0.6067, -0.4271, -0.2246,  0.2273, -0.1100, -0.2183, -0.3709, -0.1340,
         -0.1130,  0.6494,  0.6441, -0.1387,  0.2489,  0.2713, -0.0351,  1.2031],
        [ 0.2010,  0.8507,  0.6533,  0.2228,  0.3173,  0.8365,  0.6526,  0.3822,
         -0.6315, -1.2205, -0.4374, -0.2859, -0.9985,  0.1108, -0.1001,  0.5346],
        [ 0.1453,  0.4755,  0.1447, -0.2496, -0.0209,  0.4674,  0.0808, -0.2074,
         -0.5866,  0.0157,  0.1711, -0.3741, -0.3699, -0.1248,  0.1164,  0.7404],
        [-0.2268,  0.2

In [24]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5877, 0.4123, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4457, 0.2810, 0.2733, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2220, 0.7496, 0.0175, 0.0109, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0379, 0.0124, 0.0412, 0.0630, 0.8454, 0.0000, 0.0000, 0.0000],
        [0.5497, 0.2187, 0.0185, 0.0239, 0.1831, 0.0062, 0.0000, 0.0000],
        [0.2576, 0.0830, 0.0946, 0.0241, 0.1273, 0.3627, 0.0507, 0.0000],
        [0.0499, 0.1052, 0.0302, 0.0281, 0.1980, 0.2657, 0.1755, 0.1474]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [25]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2,-1) * head_size**-0.5 # What is the mathematical proof that enables this scaling

In [26]:
k.var(), q.var(), wei.var()

(tensor(1.0449), tensor(1.0700), tensor(1.0918))

In [27]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [28]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [29]:
class LayerNorm1d: # used to be BatchNorm
    def __init__(self, dim, eps=1e-5, moementum=0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
    
    def __call__ (self, x):
        # calculate the forward pass
        xmean = x.mean(1, keepdim=True) # layer mean, the columns
        xvar = x.var(1, keepdim=True) # layer var
        # print("xmean", xmean.shape)
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]
    
torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32,100) # batch size of 32 of 100 dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [30]:
# Compare batch norm - (1,100) vs layer norm - (32,1)
# See xmean shape, if all mean from all batches or just one batch - (32,1) which will be broadcasted 


In [31]:
x[:,0].mean(), x[:,0].std()

(tensor(0.1469), tensor(0.8803))

In [32]:
x[0,:].mean(), x[0,:].std() # if computed horizontally(dim=1) the mean is 0 and std is 1

(tensor(-3.5763e-09), tensor(1.0000))

In [33]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



### Full finished code, for reference

You may want to refer directly to the git repo instead though.

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters =  5000
eval_interval = 100
learning_rate = 1e-3
device = 'cude' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# -------

torch.manual_seed(1337)
# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)

# create a mapping from char to integers
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for ch,i in stoi.items()}
encode = lambda s:          [stoi[c] for c in s]    # encoder: take a string, output a list of integers
decode = lambda l: ''.join( [itos[i] for i in l])   # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(len(data)*0.9)
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters) # reset the losses
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention layer"""
    def __init__ (self, head_size):
        super().__init__()
        # learnable weights/functions for all of the embedding/channels
        self.key = nn.Linear(n_embd, head_size, bias=False) 
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) # Since not parameter, we register it as a buffer

        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x) # (B,T,H)
        q = self.query(x) # (B,T,H)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B,T,H) @ (B,H,T) --> (B,T,T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B,T,T) - self.tril[:T, :T] since we can get T<=block_size during generation
        wei = F.softmax(wei, dim=-1) # (B,T,T)
        wei = self.dropout(wei) 

        # perform the weighted aggregation of the values
        v  = self.value(x) # (B,T,H)
        out = wei @ v # (B,T,T) @ (B,T,H) --> (B,T,H)
        return out
    
class MultiHeadedAttention(nn.Module):
    """ multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)]) # the outputs of the heads will later be concatenated
        self.proj = nn.Linear(n_embd, n_embd)  # used to mix the concatenated heads
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out
    

class FeedForward(nn.Module):
    """ a simple linear layer followed by non-linearity"""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 *n_embd), # 4 times the embedding size to allow for more complex functions
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer Block: communication followed by computation"""

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadedAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = LayerNorm1d(n_embd)
        self.ln2 = LayerNorm1d(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x
    
# super simple Bigram model
class BigramLanguageModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        # each token directly read off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])  # * to unpack the list into different arguments than one single argument containing a list
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # T,C
        x = tok_emb + pos_emb # (B,T,C) - Adds position embedding to the token embedding
        x = self.blocks(x) # (B,T,C) - transformer block
        x = self.ln_f(x) # (B,T,C) - layer norm
        logits = self.lm_head(x) # (B,T,vocab_size) - linear layer

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C) # (B,T,C) -> (B*T,C) - since we don't want to deal with (B,C,T) tensors expected for cross_entropy. we instead flatten the first two dimensions
            targets = targets.view(B*T) # (B,T) -> (B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indeces in the current context
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:] # crop idx to the last block_size tokens
            logits, loss = self(idx_cond) # get the predictions
            logits = logits[:,-1,:] # becomes (B,C) - focus only on the last time step - the probabilities/logits of the next token base on the last token
            probs = F.softmax(logits, dim=-1) # (B,C) - apply softmax to get the probabilities
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1) - sample from the distribution
            idx = torch.cat((idx, idx_next), dim=1) # (B,T+1) - append sampled index to the running sequence
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr= learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on the train and val sets
    if iter % eval_interval == 0 or iter == max_iters-1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


# generate from the model
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.208705 M parameters
step 0: train loss 4.3887, val loss 4.3809
step 100: train loss 2.6180, val loss 2.6325
step 200: train loss 2.3288, val loss 2.3277
step 300: train loss 2.0652, val loss 2.0834
step 400: train loss 1.7855, val loss 1.8012
step 500: train loss 1.5381, val loss 1.5658
step 600: train loss 1.3008, val loss 1.3232
step 700: train loss 1.0785, val loss 1.1232
step 800: train loss 0.9359, val loss 0.9678
step 900: train loss 0.8212, val loss 0.8483
step 1000: train loss 0.6981, val loss 0.7334
step 1100: train loss 0.6460, val loss 0.6834
step 1200: train loss 0.6261, val loss 0.6539
step 1300: train loss 0.5150, val loss 0.5387
step 1400: train loss 0.4577, val loss 0.4831
step 1500: train loss 0.4101, val loss 0.4442
step 1600: train loss 0.4019, val loss 0.4369
step 1700: train loss 0.3458, val loss 0.3761
step 1800: train loss 0.3215, val loss 0.3507
step 1900: train loss 0.3098, val loss 0.3410
step 2000: train loss 0.2859, val loss 0.3198
step 2100: train loss 0.

In [ ]:
a = torch.randn(1,4)
print(a)
a[:,-4:]

In [ ]:
print(torch.arange(T))